In [1]:
#Import relevant libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

In [2]:
#Import data
videogames = pd.read_csv('../data/vgsales.csv')

In [3]:
# Create and display ProfileReport
profile = ProfileReport(videogames, title = "Video Game Sales 1985-2009")
profile.to_widgets()

In [4]:
videogames['Platform'].value_counts()

DS      2163
PS2     2161
PS3     1329
Wii     1325
X360    1265
PSP     1213
PS      1196
PC       960
XB       824
GBA      822
GC       556
3DS      509
PSV      413
PS4      336
N64      319
SNES     239
XOne     213
SAT      173
WiiU     143
2600     133
GB        98
NES       98
DC        52
GEN       27
NG        12
WS         6
SCD        6
3DO        3
TG16       2
PCFX       1
GG         1
Name: Platform, dtype: int64

In [5]:
# The platforms category is extremely granular and unlikely to be useful for modeling
# Here I use a dictionary to create additional columns.
# platformcat = separated out into the five major groups (nintendo, playstation, xbox, pc, and other)
# platformtype = separated out into the three major groups (console, portable, PC)

platformcatsdict = {
    'DS' : ['Nintendo', 'Portable'],
    'PS2' : ['PlayStation', 'Console'],
    'PS3' : ['PlayStation', 'Console'],
    'Wii' : ['Nintendo', 'Console'],
    'X360' : ['Xbox', 'Console'],
    'PSP' : ['PlayStation', 'Portable'],
    'PS' : ['PlayStation', 'Console'],
    'PC' : ['PC', 'PC'],
    'XB' : ['Xbox', 'Console'],
    'GBA' : ['Nintendo', 'Portable'],
    'GC' : ['Nintendo', 'Console'],
    '3DS' : ['Nintendo', 'Portable'],
    'PSV' : ['PlayStation', 'Portable'],
    'PS4' : ['PlayStation', 'Console'],
    'N64' : ['Nintendo', 'Console'],
    'SNES' : ['Nintendo', 'Console'],
    'XOne' : ['Xbox', 'Console'],
    'SAT' : ['Other', 'Console'],
    'WiiU' : ['Nintendo', 'Console'],
    '2600' : ['Other', 'Console'],
    'GB' : ['Nintendo', 'Portable'],
    'NES' : ['Nintendo', 'Console'],
    'DC' : ['Other', 'Console'],
    'GEN' : ['Other', 'Console'],
    'NG' : ['Other', 'Console'],
    'WS' : ['Other', 'Portable'],
    'SCD' : ['Other', 'Console'],
    '3DO' : ['Other', 'Console'],
    'TG16' : ['Other', 'Console'],
    'GG' : ['Other', 'Console'],
    'PCFX' : ['Other', 'Console']
}

videogames['platformcat'] = [platformcatsdict[x][0] for x in videogames['Platform']]
videogames['platformcat'].value_counts()

# Why are there so mayn more playstation games than xbox? Is it because playstation had portable consoles while xbox didn't?
# or is there a bias in the way the data was pulled?

videogames['platformtype'] = [platformcatsdict[x][1] for x in videogames['Platform']]
videogames['platformtype'].value_counts()

Console     10414
Portable     5224
PC            960
Name: platformtype, dtype: int64

In [6]:
videogames['Year'] = videogames['Year'].astype('int', errors = 'ignore')
videogames['Year'].value_counts()

2009.0    1431
2008.0    1428
2010.0    1259
2007.0    1202
2011.0    1139
2006.0    1008
2005.0     941
2002.0     829
2003.0     775
2004.0     763
2012.0     657
2015.0     614
2014.0     582
2013.0     546
2001.0     482
1998.0     379
2000.0     349
2016.0     344
1999.0     338
1997.0     289
1996.0     263
1995.0     219
1994.0     121
1993.0      60
1981.0      46
1992.0      43
1991.0      41
1982.0      36
1986.0      21
1989.0      17
1983.0      17
1990.0      16
1987.0      16
1988.0      15
1985.0      14
1984.0      14
1980.0       9
2017.0       3
2020.0       1
Name: Year, dtype: int64

In [7]:
# These titles seem to have erroneous release dates, or are in a release year that isn't a part of the main dataset.
# Including them may impact our models, so they will be removed. 
videogames.drop(videogames[videogames['Year'].isin([2017, 2020])].index, inplace = True)
videogames['Year'].value_counts()

2009.0    1431
2008.0    1428
2010.0    1259
2007.0    1202
2011.0    1139
2006.0    1008
2005.0     941
2002.0     829
2003.0     775
2004.0     763
2012.0     657
2015.0     614
2014.0     582
2013.0     546
2001.0     482
1998.0     379
2000.0     349
2016.0     344
1999.0     338
1997.0     289
1996.0     263
1995.0     219
1994.0     121
1993.0      60
1981.0      46
1992.0      43
1991.0      41
1982.0      36
1986.0      21
1989.0      17
1983.0      17
1990.0      16
1987.0      16
1988.0      15
1985.0      14
1984.0      14
1980.0       9
Name: Year, dtype: int64

In [8]:
# Take a look at which video games are missing the year indicator.
# Some of these are sports games that have a year in their title. We can fix these. 
# For example, any sports game that has 2005 in the title is almost always released in 2004

videogames[videogames['Year'].isna() == True]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,platformcat,platformtype
179,180,Madden NFL 2004,PS2,NaN,Sports,Electronic Arts,4.26,0.26,0.01,0.71,5.23,PlayStation,Console
377,378,FIFA Soccer 2004,PS2,NaN,Sports,Electronic Arts,0.59,2.36,0.04,0.51,3.49,PlayStation,Console
431,432,LEGO Batman: The Videogame,Wii,NaN,Action,Warner Bros. Interactive Entertainment,1.86,1.02,0.00,0.29,3.17,Nintendo,Console
470,471,wwe Smackdown vs. Raw 2006,PS2,NaN,Fighting,NaN,1.57,1.02,0.00,0.41,3.00,PlayStation,Console
607,608,Space Invaders,2600,NaN,Shooter,Atari,2.36,0.14,0.00,0.03,2.53,Other,Console
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16307,16310,Freaky Flyers,GC,NaN,Racing,Unknown,0.01,0.00,0.00,0.00,0.01,Nintendo,Console
16327,16330,Inversion,PC,NaN,Shooter,Namco Bandai Games,0.01,0.00,0.00,0.00,0.01,PC,PC
16366,16369,Hakuouki: Shinsengumi Kitan,PS3,NaN,Adventure,Unknown,0.01,0.00,0.00,0.00,0.01,PlayStation,Console
16427,16430,Virtua Quest,GC,NaN,Role-Playing,Unknown,0.01,0.00,0.00,0.00,0.01,Nintendo,Console


In [9]:
# Define regular expression to find video games with years in their name
# Selects a group of characters that begins with 19 or 20 and is followed by two other numbers
expression = '(19|20\d{2})'

# Use rank as a unique identifier and create a list of the records that will be changed so that I can check if the transformation works
checklist = list(videogames.loc[(videogames['Name'].str.contains(expression) & videogames['Year'].isna() == True), ['Name', 'Year', 'Rank']]['Rank'].values)
print(checklist)



[180, 378, 471, 1651, 2588, 3503, 4799, 5164, 5242, 5671, 5903, 8931, 12924, 13872, 15742, 15868, 16060]


C:\Users\atyou\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [10]:
# Extract series to replace missing years
replacement_years = videogames[(videogames['Name'].str.contains(expression) & videogames['Year'].isna() == True)]['Name'].str.extract(expression).astype('int')
replacement_years = replacement_years - 1
replacement_years = replacement_years.values
replacement_years = replacement_years.reshape(-1,)
replacement_years.shape



C:\Users\atyou\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


(17,)

In [11]:
# Reassign missing years to the new replacement_years list
videogames.loc[(videogames['Name'].str.contains(expression) & videogames['Year'].isna() == True), 'Year'] = replacement_years

C:\Users\atyou\Anaconda3\lib\site-packages\pandas\core\strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [12]:
# Verify that the years were changed successfully
videogames[videogames['Rank'].isin(checklist)]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,platformcat,platformtype
179,180,Madden NFL 2004,PS2,2003.0,Sports,Electronic Arts,4.26,0.26,0.01,0.71,5.23,PlayStation,Console
377,378,FIFA Soccer 2004,PS2,2003.0,Sports,Electronic Arts,0.59,2.36,0.04,0.51,3.49,PlayStation,Console
470,471,wwe Smackdown vs. Raw 2006,PS2,2005.0,Fighting,NaN,1.57,1.02,0.00,0.41,3.00,PlayStation,Console
1649,1651,NASCAR Thunder 2003,PS2,2002.0,Racing,Unknown,0.60,0.46,0.00,0.16,1.22,PlayStation,Console
2586,2588,PES 2009: Pro Evolution Soccer,PSP,2008.0,Sports,Konami Digital Entertainment,0.04,0.33,0.26,0.17,0.80,PlayStation,Portable
3501,3503,Madden NFL 2002,XB,2001.0,Sports,Unknown,0.53,0.02,0.00,0.03,0.58,Xbox,Console
4797,4799,NFL GameDay 2003,PS2,2002.0,Sports,Unknown,0.20,0.15,0.00,0.05,0.40,PlayStation,Console
5162,5164,NBA Live 2003,XB,2002.0,Sports,Electronic Arts,0.31,0.04,0.00,0.01,0.36,Xbox,Console
5240,5242,Tomb Raider (2013),PC,2012.0,Action,Square Enix,0.06,0.25,0.00,0.05,0.36,PC,PC
5669,5671,All-Star Baseball 2005,PS2,2004.0,Sports,Unknown,0.16,0.12,0.00,0.04,0.32,PlayStation,Console


In [13]:
# Remove all other rows that have a missing year
videogames.drop(videogames[videogames['Year'].isna() == True].index, inplace = True)
videogames.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16340 entries, 0 to 16597
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16340 non-null  int64  
 1   Name          16340 non-null  object 
 2   Platform      16340 non-null  object 
 3   Year          16340 non-null  float64
 4   Genre         16340 non-null  object 
 5   Publisher     16303 non-null  object 
 6   NA_Sales      16340 non-null  float64
 7   EU_Sales      16340 non-null  float64
 8   JP_Sales      16340 non-null  float64
 9   Other_Sales   16340 non-null  float64
 10  Global_Sales  16340 non-null  float64
 11  platformcat   16340 non-null  object 
 12  platformtype  16340 non-null  object 
dtypes: float64(6), int64(1), object(6)
memory usage: 1.7+ MB


In [14]:
# The remaining missing values are in the Publisher column, lets take a look at them:
# They all appear to be relatively small games, so lets set their publisher to 'Other'
videogames.loc[videogames['Publisher'].isna() == True, 'Publisher'] = 'Other'
videogames.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16340 entries, 0 to 16597
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          16340 non-null  int64  
 1   Name          16340 non-null  object 
 2   Platform      16340 non-null  object 
 3   Year          16340 non-null  float64
 4   Genre         16340 non-null  object 
 5   Publisher     16340 non-null  object 
 6   NA_Sales      16340 non-null  float64
 7   EU_Sales      16340 non-null  float64
 8   JP_Sales      16340 non-null  float64
 9   Other_Sales   16340 non-null  float64
 10  Global_Sales  16340 non-null  float64
 11  platformcat   16340 non-null  object 
 12  platformtype  16340 non-null  object 
dtypes: float64(6), int64(1), object(6)
memory usage: 1.7+ MB


In [15]:
#Lets create two new ways to deal with the publisher variable. We can decide which one is better later

#create new column that is the count of titles released by that publisher
publisher_title_count = videogames.groupby('Publisher').count().to_dict()
videogames['publisher_title_count'] = [publisher_title_count['Rank'][x] for x in videogames['Publisher']]
videogames['publisher_title_count'].describe()

count    16340.000000
mean       497.728886
std        426.450433
min          1.000000
25%         73.000000
50%        376.000000
75%        917.000000
max       1343.000000
Name: publisher_title_count, dtype: float64

In [16]:
# create a new column that is the publisher name for any publishers with over X amount of titles released, 
# and 'Other' for anything else
# I'll use the 50th percentile as X for now (376)

for publisher in videogames['Publisher'].unique():
    if list(videogames.loc[videogames['Publisher'] == publisher, 'publisher_title_count'])[0] >= 376:
        videogames.loc[videogames['Publisher'] == publisher, 'Publisher_2'] = publisher
    else:
        videogames.loc[videogames['Publisher'] == publisher, 'Publisher_2'] = 'Other'

In [17]:
# This version of 'Publisher only has 12 different categories, compared to ~575
# it may be worth lowering the threshhold to capture more of the large studios

videogames['Publisher_2'].value_counts()

Other                           7853
Electronic Arts                 1343
Activision                       966
Namco Bandai Games               928
Ubisoft                          917
Konami Digital Entertainment     824
THQ                              712
Nintendo                         696
Sony Computer Entertainment      682
Sega                             631
Take-Two Interactive             412
Capcom                           376
Name: Publisher_2, dtype: int64

In [18]:
# This code is to create two new columns:
# 1. global population for year
# 2. global sales as % of global population

# load data
global_population = pd.read_excel('../data/worldpopulation.xlsx', sheet_name = 'ay edited')

In [19]:
# convert to dict
global_population = global_population.set_index('Year')
global_population = global_population.to_dict()
global_population

{'World Population': {1980: 4458003.466,
  1981: 4536996.619,
  1982: 4617386.526,
  1983: 4699569.187,
  1984: 4784011.517,
  1985: 4870921.666,
  1986: 4960568.0,
  1987: 5052521.998,
  1988: 5145425.994,
  1989: 5237441.434,
  1990: 5327231.04100001,
  1991: 5414289.383,
  1992: 5498919.893,
  1993: 5581597.598,
  1994: 5663150.428,
  1995: 5744212.93,
  1996: 5824891.93100001,
  1997: 5905045.647,
  1998: 5984794.075,
  1999: 6064239.03300001,
  2000: 6143493.806,
  2001: 6222626.531,
  2002: 6301773.17199999,
  2003: 6381185.141,
  2004: 6461159.391,
  2005: 6541906.956,
  2006: 6623517.917,
  2007: 6705946.643,
  2008: 6789088.672,
  2009: 6872766.988,
  2010: 6956823.588,
  2011: 7041194.16799999,
  2012: 7125827.957,
  2013: 7210582.041,
  2014: 7295290.759,
  2015: 7379796.967,
  2016: 7464021.934,
  2017: 7547858.9,
  2018: 7631091.113,
  2019: 7713468.20500001,
  2020: 7794798.729}}

In [20]:
# Create first new variable
videogames['globalpop'] = [global_population['World Population'][x] / 1000 for x in videogames['Year']]
videogames.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,platformcat,platformtype,publisher_title_count,Publisher_2,globalpop
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Nintendo,Console,696,Nintendo,6623.517917
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,Nintendo,Console,696,Nintendo,4870.921666
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Nintendo,Console,696,Nintendo,6789.088672
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,Nintendo,Console,696,Nintendo,6872.766988
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,Nintendo,Portable,696,Nintendo,5824.891931


In [21]:
# Create second new variable
videogames['global_sales_as_pct_of_pop'] = (videogames['Global_Sales'] / videogames['globalpop'] * 100)
videogames.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,platformcat,platformtype,publisher_title_count,Publisher_2,globalpop,global_sales_as_pct_of_pop
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74,Nintendo,Console,696,Nintendo,6623.517917,1.249185
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,Nintendo,Console,696,Nintendo,4870.921666,0.826127
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82,Nintendo,Console,696,Nintendo,6789.088672,0.527611
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00,Nintendo,Console,696,Nintendo,6872.766988,0.480156
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,Nintendo,Portable,696,Nintendo,5824.891931,0.538551


In [22]:
# Re-run profile report to analyze clean dataset with newly created features
profile = ProfileReport(videogames, title = "Video Game Sales 1985-2009")
profile.to_widgets()